<a href="https://colab.research.google.com/github/riverdogcabin/PSDS4900/blob/main/wunderground_pws_by_geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
import json
from pprint import pprint
import pandas as pd
from datetime import datetime
from time import sleep

In [15]:
# Set up API call URL
# weatherreadings = {}
url = 'https://api.weather.com/v3/location/near?geocode={geocode}&product=pws&format=json&apiKey={apikey}'
# url = 'https://api.weather.com/v2/pws/observations/hourly/7day?stationId={stationid}&format=json&units={units}&apiKey={apikey}&numericPrecision={numeric_precision}'
# url = "https://api.weather.com/v2/pws/observations/current?stationId=REPLACESTATIONID&format=json&units=REPLACEUNITS&numericPrecision=decimal&apiKey=REPLACEAPI"


In [16]:
# Get Config Data from config.json
with open('/content/drive/MyDrive/Capstone/PSDS4900/config.json') as configuration:
  config = json.load(configuration).get('WU') #config for WeatherUnderground
  apikey = config["apikey"]

In [17]:
url = url.format(geocode = '{geocode}',apikey=apikey)
# print(url)
points_file = '/content/drive/MyDrive/Capstone/PSDS4900/points.txt'
station_file = '/content/drive/MyDrive/Capstone/PSDS4900/wu_stations.txt'
# drop_columns = ['lat','lon','units','tz','obsTimeUtc','obsTimeLocal']

In [18]:
# JSON comes back with measurements nested inside 'metrics', need to flatten
def flatten_wu_result(wu_result,units='metric'):
  try:
    observation = wu_result.pop(units)
    wu_result.update(observation)
    wu_result.update({'units':units})
  except:
    return wu_result

In [19]:
# function to get the JSON data from Weather Underground
def get_wu_data(point,url):
  url = url.format(geocode=point)
  print(url)
  print("Retrieving data for " + point,end=': ')
  response = requests.get(url) # need to add error handling for invalid data

  if response.status_code == 200: # If the response code is 200 (sucessful)
      print("Success")
      weatherdata = response.json()
      # pprint(weatherdata)
      # for observation in weatherdata.get('observations'):
      #   flatten_wu_result(observation)
      return weatherdata.get('location')
  elif response.status_code == 204: # 204 returns a blank response so inform the user
      print("API Error 204: No Data Found for specific query.")
      exit()

  elif response.status_code >200 & response.status_code <1000: # Handles all other error code numbers
      result = response.json()
      try:
          print("API Error " + str(response.status_code) + ": " + result["errors"][0]["error"]["message"])
      except: # sometimesthe error messae is in a different nest
          print("API Error " + str(response.status_code) + ": " + result["errors"][0]["message"])
      exit()

  else: # Catch any other errors
      print("Unspecified error")
      print(response.text)

In [20]:
def get_points(pointsfile):
  point_list = []
  with open(pointsfile) as infile:
    for l in infile.readlines():
      if(l.find('#') == -1):
        point_list.append(l.strip().replace(' ',''))
  return point_list

In [21]:
points_list = get_points(points_file)
# print(points_list)

In [22]:
stations_df = pd.DataFrame()
for point in points_list:
  stations_df = stations_df.append(pd.DataFrame(get_wu_data(point,url)))
  sleep(1)

https://api.weather.com/v3/location/near?geocode=39.546313715101626,-107.32209991790515&product=pws&format=json&apiKey=84daca7ca4a844b59aca7ca4a814b54b
Retrieving data for 39.546313715101626,-107.32209991790515: Success
https://api.weather.com/v3/location/near?geocode=40.4871685058606,-106.83251810905537&product=pws&format=json&apiKey=84daca7ca4a844b59aca7ca4a814b54b
Retrieving data for 40.4871685058606,-106.83251810905537: Success
https://api.weather.com/v3/location/near?geocode=39.267192804381075,-103.69377974723669&product=pws&format=json&apiKey=84daca7ca4a844b59aca7ca4a814b54b
Retrieving data for 39.267192804381075,-103.69377974723669: Success
https://api.weather.com/v3/location/near?geocode=39.09301586109839,-104.8717627506403&product=pws&format=json&apiKey=84daca7ca4a844b59aca7ca4a814b54b
Retrieving data for 39.09301586109839,-104.8717627506403: Success


In [23]:
stations_df

,stationName,stationId,qcStatus,updateTimeUtc,partnerId,latitude,longitude,distanceKm,distanceMi
0,Glenwood Spgs,KCOGLENW48,1,1617591039,wu,39.54700,-107.32000,0.20,0.13
1,Glenwood Springs,KCOGLENW49,1,1617610328,wu,39.54500,-107.32000,0.20,0.13
2,Glenwood Springs,KCOGLENW24,1,1617641111,wu,39.55002,-107.32201,0.45,0.28
3,Glenwood Springs,KCOGLENW37,-1,1611865200,wu,39.52200,-107.32600,2.69,1.67
4,Glenwood Springs,KCOGLENW30,1,1617637114,wu,39.52129,-107.32260,2.75,1.71
5,Glenwood Springs,KCOGLENW55,1,1617652196,wu,39.51316,-107.31641,3.68,2.29
6,Glenwood Springs,KCOGLENW6,1,1617576950,wu,39.51261,-107.31355,3.78,2.35
7,Glenwood Springs,KCOGLENW23,-1,1602094986,wu,39.52408,-107.35907,4.01,2.49
8,Glenwood Springs,KCOGLENW12,0,1614240624,wu,39.50914,-107.31725,4.12,2.56
9,Glenwood Springs,KCOGLENW44,-1,1617651511,wu,39.51271,-107.34833,4.34,2.69


In [24]:
stations_df = stations_df[stations_df.qcStatus == 1]
# stations_df.to_csv('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/{}_stations.csv'.format(datetime.now().date().strftime('%Y%m%d')),index=False)
# print('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/{}_stations.csv written\n'.format(datetime.now().date().strftime('%Y%m%d')))  

In [27]:
# stations_df.stationId

0     KCOGLENW48
1     KCOGLENW49
2     KCOGLENW24
4     KCOGLENW30
5     KCOGLENW55
6      KCOGLENW6
0     KCOSTEAM64
1     KCOSTEAM55
2     KCOSTEAM96
3     KCOSTEAM28
4     KCOSTEAM38
6     KCOSTEAM21
8     KCOSTEAM93
0    KCOMONUM250
4     KCOMONUM50
7     KCOMONUM86
8     KCOMONUM77
Name: stationId, dtype: object

In [30]:
with open(station_file,mode='a') as outfile:
  outfile.write('\n'.join(stations_df.stationId.to_list()))